<a href="https://colab.research.google.com/github/takedatmh/toyama/blob/main/Toyama_Uni_2_8_FineTuning_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf ~/.cache/huggingface/datasets
!rm -rf ~/.cache/huggingface/hub

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# LLMファインチューニングに必要なライブラリ群
!pip install -q \
  transformers \
  datasets \
  accelerate \
  bitsandbytes \
  peft \
  sentencepiece \
  scipy \
  evaluate \
  huggingface-hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.4 MB/s eta 0:00:00


In [4]:
pip install -U fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.9.0 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.


In [5]:
# Downgrade fsspec to resolve the conflict
!pip install fsspec==2025.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.9.0
    Uninstalling fsspec-2025.9.0:
      Successfully uninstalled fsspec-2025.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.3.2 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
from huggingface_hub import login

# デバイス確認（T4 GPU が見えるか）
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")

# 1. モデルとトークナイザの準備
# model_name = "rinna/japanese-gpt2-small"
model_name = "elyza/ELYZA-japanese-Llama-2-7b"

# 8bit量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16  # T4 GPU に最適
)

# # Instead, move the model to the device explicitly after loading.
# model.to(device)

# # 2. LoRAの設定 GPT2系のrinna/japanese-gpt2-smallの場合（PEFT）
# lora_config = LoraConfig(
#     r=8,
#     lora_alpha=32,
#     target_modules=["c_proj", "c_attn"],  # Changed target modules to match GPT2 architecture
#     lora_dropout=0.05,
#     bias="none",
#     task_type=TaskType.CAUSAL_LM
# )

# 2. LoRAの設定 Llama2系 ELYZAの場合
  # モジュール名	用途・場所
  # q_proj	AttentionのQuery部分
  # k_proj	AttentionのKey部分
  # v_proj	AttentionのValue部分
  # o_proj	Attentionの出力変換部分
  # gate_proj	MLP(FFN)部のGating層
  # up_proj	MLP(FFN)部の中間変換層
  # down_proj	MLP(FFN)部の出力層
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention
        "gate_proj", "up_proj", "down_proj"      # MLP
    ]
)

model = get_peft_model(model, lora_config)

# 3. データセット読み込みと前処理 スプリット指定は使わない（問題の回避）
# dataset = load_dataset("ag_news", split="train")
dataset = load_dataset("izumi-lab/llm-japanese-dataset", split="train")
dataset_small = dataset.select(range(1000))

def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset_small.map(tokenize_fn, batched=True)


# 4. トレーニング引数の設定
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1, #本当は3回ぐらい回したい
    fp16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

# 5. トレーナー定義と学習開始
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()


Running on cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss
10,2.555300
20,2.177200
30,2.107900
40,2.097800
50,1.948000
60,2.037000
70,2.053500
80,1.968200
90,2.110500
100,2.008100


TrainOutput(global_step=125, training_loss=2.0633645706176758, metrics={'train_runtime': 136.9148, 'train_samples_per_second': 7.304, 'train_steps_per_second': 0.913, 'total_flos': 5089791049728000.0, 'train_loss': 2.0633645706176758, 'epoch': 1.0})

# Fine-Tuning後のモデルを利用して推論(Chat)を実行


In [9]:
from transformers import GenerationConfig

model.eval()

# より構造化されたプロンプト（LoRAで学習している形式に合わせること）
prompt = "このニュースを分類してください：「株式市場が好調な企業業績を受けて上昇」"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 推論設定
generation_config = GenerationConfig(
    max_new_tokens=128,
    do_sample=True,
    top_p=0.95,
    temperature=0.7,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id
)

with torch.no_grad():
    output = model.generate(
        **inputs,
        generation_config=generation_config
    )

# 出力をプロンプトと分離して表示
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("回答:", generated_text.replace(prompt, "").strip())

🧠 回答: （106）、「国内景気が回復する見通しで上昇」（58）、「金利上昇により企業の資金調達コストが高まる」（43）、「景気後退の影響から下落」（37） 自民党の石破茂政調会長は29日午前の記者会見で、安


#日本語データセット版